In [17]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
#Set Connection parameters
USER = 'aacuser'
PASS = 'SNHU2025'
HOST = 'nv-desktop-services.apporto.com'
PORT = 30167
DB = 'AAC'
COL = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(USER,PASS,HOST,PORT,DB,COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash("Austin Thompson - SNHU CS 340")
# Encode the logo image
image_filename = 'AACLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# app layout
app.layout = html.Div([
    # Header section with logo made to be side by side
    html.Div(style={'display': 'flex','alignItems': 'center','justifyContent': 'center','gap': '50px'},
        children=[
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'width': '150px'}
                ),
            html.Div([
                html.H1('CS-340 Dashboard - Grazioso Salvare', style={'margin': '0'}),
                html.H2('Austin Thompson', style={'margin': '0', 'fontWeight': 'normal'})
            ])
        
        ]
    ),
    html.Hr(),
    # Radio Buttons by rescue type
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
            ],
            value='all',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data = df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # Sets up the query filters for each rescue type
    if filter_type == 'water':
        query = {
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        query = {
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        query = {
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        # If all is selected, no filter
        query = {}
    # Read the data
    results = pd.DataFrame(list(db.read(query)))
    # Remove ID columns if needed
    if '_id' in results.columns:
        results.drop(columns=['_id'], inplace=True)
    # For 'all' display only the top 5 so it looks better
    if filter_type == 'all' and not results.empty:
        top_breeds = results['breed'].value_counts().nlargest(5).index
        results = results[results['breed'].isin(top_breeds)]
    # return the records as dictionary list
    return results.to_dict('records')
        
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')])
def update_graphs(viewData, filter_type):
    # Placeholder text if no data is available
    if viewData is None or len(viewData) == 0:
        return[html.P("No data to display in chart")]
    dff = pd.DataFrame.from_dict(viewData)
    # If it is 'all' change the title
    if filter_type == 'all':
        fig = px.pie(dff, names='breed', title='Top 5 Breeds')
    else:
        fig = px.pie(dff, names='breed', title='Distribution of Preferred Breeds')
    return [dcc.Graph(figure = fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    #Convert table data into a dataframe for easier indexing
    dff = pd.DataFrame.from_dict(viewData)
    
    # If no data is available, display a blank map
    if dff.empty:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, 
                      center=[30.75,-97.48], zoom=10, children=[dl.TileLayer()])
    # Get the index of the selected row, set 0 if nothing is selected
    row = index[0] if index else 0
    #return a map which is centered on Austin with a marker for the selected animal
    return dl.Map(style={'width': '1000px', 'height': '500px'}, 
                      center=[30.75,-97.48],
                      zoom=10,
                      children=[dl.TileLayer(id="base-layer-id"),
                            dl.Marker(
                                position=[dff.iloc[row, 13], dff.iloc[row, 14]], # coordinates longitude and latitude
                                children=[dl.Tooltip(dff.iloc[row, 4]), # Tooltip shows animal breed
                                # Animal's name popup
                                dl.Popup([
                                    html.H1("Animal Name"),
                                    html.P(dff.iloc[row, 9])
                                ])]
                            )])


app.run_server(debug=True)


Dash app running on http://127.0.0.1:29475/
